In [8]:
import torch
import numpy as np
from torch_sparse import SparseTensor, spspmm

In [78]:
edge_index = torch.LongTensor([[0, 0, 0, 1, 3, 3], [0, 1, 2, 0, 1, 2]])
nodes = torch.max(edge_index).item() + 1
sp1 = SparseTensor.from_edge_index(edge_index, sparse_sizes=(nodes, nodes))
print(sp1)
vl1 = torch.ones((edge_index.shape[1]))
np.random.seed(1)
dense = torch.from_numpy(np.random.randint(2, size=(4, 4), dtype=np.int64))
sp2 = SparseTensor.from_dense(dense)
vl2 = torch.ones((torch.count_nonzero(dense)))
eix, val = spspmm(edge_index, vl1, edge_index, vl1, 4, 4, 4)


SparseTensor(row=tensor([0, 0, 0, 1, 3, 3]),
             col=tensor([0, 1, 2, 0, 1, 2]),
             size=(4, 4), nnz=6, density=37.50%)


In [81]:
print(ei, val)
ei = eix[0]
ei_ = torch.cat([ei[0:1], ei[:-1]])

cutpoints = torch.nonzero(ei - ei_).squeeze().tolist()
cutpoints = [0] + cutpoints + [ei.shape[0]]
cutpoints

tensor([0, 0, 0, 1, 1, 1, 3]) tensor([2., 1., 1., 1., 1., 1., 1.])


[0, 3, 6, 7]

In [65]:
adj_raw = [(ei[:, start:end], val[start:end]) for start, end in zip(cutpoints[:-1], cutpoints[1:])]
adj_raw

[(tensor([[0, 0, 0],
          [0, 1, 2]]),
  tensor([2., 1., 1.])),
 (tensor([[1, 1, 1],
          [0, 1, 2]]),
  tensor([1., 1., 1.])),
 (tensor([[3],
          [0]]),
  tensor([1.]))]

In [75]:
mean_value = [torch.mean(value) / 1. for data, value in adj_raw]
adj_selected_raw = [data[:, value<mean] for (data, value), mean in zip(adj_raw, mean_value)]
adj_selected = torch.cat(adj_selected_raw, dim=-1)

In [77]:
SparseTensor.from_edge_index(adj_selected, sparse_sizes=(nodes, nodes))

SparseTensor(row=tensor([0, 0]),
             col=tensor([1, 2]),
             size=(4, 4), nnz=2, density=12.50%)